<div style="text-align: center;">
<h1>NFL Elo Rating System Explained</h1>
</div>

## 1. Overview

- Every team starts with a base Elo rating (e.g., 1500).
- After each game, Elo ratings are updated based on:
  - Who won/lost (or tied)
  - The expected outcome (based on pre-game Elo)
  - The margin of victory
  - Home field advantage
- Upsets and blowouts cause bigger Elo changes.
- The bigger the surprise or the margin, the bigger the Elo change.

<br></br>

---

## 2. Step-by-Step

We'll walk through the Elo update for a single game, with formulas and Python code for each step.

### 2.1. Expected Score Calculation

The expected score is the probability each team has to win, based on their Elo ratings and home field advantage.

<br></br>

$$
\text{expected\_home} = \frac{1}{1 + 10^{\frac{(\text{away\_elo} - (\text{home\_elo} + \text{hfa}))}{400}}}
$$

$$
\text{expected\_away} = \frac{1}{1 + 10^{\frac{((\text{home\_elo} + \text{hfa}) - \text{away\_elo})}{400}}}
$$


In [ ]:
# Example Elo ratings and home field advantage
home_elo = 1500
away_elo = 1500
hfa = 2.5  # Home field advantage

expected_home = 1 / (1 + 10 ** ((away_elo - (home_elo + hfa)) / 400))
expected_away = 1 / (1 + 10 ** (((home_elo + hfa) - away_elo) / 400))

print(f"Expected home win probability: {expected_home:.3f}")
print(f"Expected away win probability: {expected_away:.3f}")


### 2.2. Actual Result and Margin

- If home team wins: `actual_home = 1`, `actual_away = 0`
- If away team wins: `actual_home = 0`, `actual_away = 1`
- If tie: `actual_home = 0.5`, `actual_away = 0.5`

The margin of victory is used to scale the Elo change.


In [ ]:
# Example game result
home_score = 24
away_score = 17

if home_score > away_score:
    actual_home, actual_away = 1, 0
    score_diff = home_score - away_score
elif away_score > home_score:
    actual_home, actual_away = 0, 1
    score_diff = away_score - home_score
else:
    actual_home, actual_away = 0.5, 0.5
    score_diff = 0

print(f"Actual home: {actual_home}, Actual away: {actual_away}, Margin: {score_diff}")


### 2.3. K-Factor Adjustment (Margin of Victory)

The K-factor is scaled up for bigger wins:

<br></br>

$$
\text{k\_adj\_factor} = 1 + \frac{\log(|\text{score\_diff}| + 1)}{10}
$$

In [ ]:
import numpy as np
k_adj_factor = 1 + np.log(abs(score_diff) + 1) / 10
print(f"K-factor adjustment: {k_adj_factor:.3f}")

### 2.4. Elo Update Step

The Elo ratings are updated for both teams:

<br></br>

$$
\text{new\_elo} = \text{old\_elo} + K \times \text{k\_adj\_factor} \times (\text{actual} - \text{expected})
$$


In [ ]:
K = 20  # base K-factor
new_home_elo = home_elo + K * k_adj_factor * (actual_home - expected_home)
new_away_elo = away_elo + K * k_adj_factor * (actual_away - expected_away)

print(f"New home Elo: {new_home_elo:.2f}")
print(f"New away Elo: {new_away_elo:.2f}")
print(f"Home Elo change: {new_home_elo - home_elo:+.2f}")
print(f"Away Elo change: {new_away_elo - away_elo:+.2f}")
